In [10]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

# Distance matrix and problem parameters
distance_matrix = [
    [0, 20, 25, 35, 65, 90, 85, 80, 86, 25, 35, 20, 44, 35, 82],  # Heathrow
    [20, 0, 15, 35, 60, 55, 57, 85, 90, 25, 35, 30, 37, 20, 40],  # Harrow
    [25, 15, 0, 30, 50, 70, 55, 50, 65, 10, 25, 15, 24, 20, 90],  # Ealing
    [35, 35, 30, 0, 45, 60, 53, 55, 47, 12, 22, 20, 12, 10, 21],  # Holborn
    [65, 60, 50, 45, 0, 46, 15, 45, 75, 25, 11, 19, 15, 25, 25],  # Sutton
    [90, 55, 70, 60, 46, 0, 15, 25, 45, 65, 53, 43, 63, 70, 27],  # Dartford
    [85, 57, 55, 53, 15, 15, 0, 17, 25, 41, 25, 33, 27, 45, 30],  # Bromley
    [80, 85, 50, 55, 45, 25, 17, 0, 25, 40, 34, 32, 20, 30, 10],  # Greenwich
    [86, 90, 65, 47, 75, 45, 25, 25, 0, 65, 70, 72, 61, 45, 13],  # Barking
    [25, 25, 10, 12, 25, 65, 41, 40, 65, 0, 20, 8, 7, 15, 25],    # Hammersmith
    [35, 35, 25, 22, 11, 53, 25, 34, 70, 20, 0, 5, 12, 45, 65],   # Kingston
    [20, 30, 15, 20, 19, 43, 33, 32, 72, 8, 5, 0, 14, 34, 56],    # Richmond
    [44, 37, 24, 12, 15, 63, 27, 20, 61, 7, 12, 14, 0, 30, 40],   # Battersea
    [35, 20, 20, 10, 25, 70, 45, 30, 45, 15, 45, 34, 30, 0, 27],  # Islington
    [82, 40, 90, 21, 25, 27, 30, 10, 13, 25, 65, 56, 40, 27, 0]   # Woolwich
]

num_cities = len(distance_matrix)
num_vehicles = 6

# Decision variables
used = [solver.BoolVar(f'vehicle_{i}_used') for i in range(num_vehicles)]
path = [[[solver.BoolVar(f'vehicle_{k}_from_{i}_to_{j}') for j in range(num_cities)] for i in range(num_cities)] for k in range(num_vehicles)]
visit = [[solver.BoolVar(f'vehicle_{k}_visits_city_{i}') for i in range(num_cities)] for k in range(num_vehicles)]
u = [[solver.NumVar(0,num_cities, f'position_of_{i}_in_vehicle_{k}') for i in range(num_cities)] for k in range(num_vehicles)]


# Constraints
for k in range(num_vehicles):
    for i in range(num_cities):
        
        for j in range(num_cities):
            solver.Add(path[k][j][j] == 0)  # Ensure no self-loops
            solver.Add(visit[k][j] <= used[k])  # Vehicle k visits city j only if used[k] is 1
        solver.Add(sum(path[k][j][i] for j in range(num_cities)) == visit[k][i])
        solver.Add(sum(path[k][i][j] for j in range(num_cities)) == visit[k][i])  # Ensure exactly one inbound and one outbound edge for each city

    solver.Add(sum(distance_matrix[i][j] * path[k][i][j] for i in range(num_cities) for j in range(num_cities)) <= 120)  # Capacity constraint for vehicle k

# Ensure each city is visited exactly once by one vehicle
for i in range(num_cities):
    if i == 0:
        solver.Add(sum(visit[k][i] - used[k] for k in range(num_vehicles))==0)
    else:
        solver.Add(sum(visit[k][i] for k in range(num_vehicles)) == 1)
for k in range(num_vehicles):
    for i in range(1,num_cities):
        solver.Add(u[k][0]==0)
        for j in range(1,num_cities):
            if i!=j:
                solver.Add(u[k][i]-u[k][j]+(num_cities-1)*path[k][i][j]<=num_cities-2)
# Subtour elimination constraints
for i in range(6):
    for j in range(6):
        if i<=j:
            solver.Add(sum(visit[i][k] for k in range(num_cities))>=sum(visit[j][k] for k in range(num_cities)))
# Objective function
solver.Minimize(sum(used[k] for k in range(num_vehicles)))
 # Sets a time limit of 60 seconds (adjust as needed)

# Solve and print solution
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for k in range(num_vehicles):
        if used[k].solution_value() >=0.5:
            print(f'Route for vehicle {k}:')
            route = []
            for i in range(num_cities):
                for j in range(num_cities):
                    if path[k][i][j].solution_value() >= 0.5:
                        route.append((i, j))
            print(route)
else:
    print('The problem does not have an optimal solution.')


The problem does not have an optimal solution.
